In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.sparse as sparse

In [2]:
dense_linear = nn.Linear(1024, 1000)
sparse_linear = nn.Linear(1024, 1000)
dense_linear

Linear(in_features=1024, out_features=1000, bias=True)

In [3]:
torch.count_nonzero(sparse_linear.weight)

tensor(1024000)

In [4]:
sparsity=0.99
idx = torch.randperm(n=sparse_linear.weight.numel())
non_zero_idx = idx[int(len(idx)*(1-0.99)):]
with torch.no_grad():
    
    w = sparse_linear.weight
    w  = w.flatten()
    print(sparse_linear.weight.shape)
    w[non_zero_idx]=0
    w = w.reshape(dense_linear.weight.shape)
    print(sparse_linear.weight.shape)





torch.Size([1000, 1024])
torch.Size([1000, 1024])


In [5]:
from datetime import datetime
input = torch.rand(size=(64, 1024))

with torch.no_grad():
    start = datetime.now()
    for _ in range(100):
        dense_linear(input)
    end = datetime.now() - start
    print(end)
    

0:00:00.055601


In [40]:
dense_linear.weight.shape

torch.Size([1000, 1024])

In [41]:
input.shape

torch.Size([64, 1024])

In [45]:
(input@dense_linear.weight.T).shape

torch.Size([64, 1000])

In [84]:
sparse_linear.weight.data.T

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [108]:
import copy
class SparseLinear(nn.Module):
    
    def __init__(self, linear_layer, bs):   
        super().__init__()
        self.weights = linear_layer.weight.T.to_sparse_coo()
        # self.weights = copy.deepcopy(linear_layer.weight.T.to_sparse_coo())
        self.bias = nn.Parameter(linear_layer.bias.expand(size=(bs, *linear_layer.bias.shape)))
        # self.bias = linear_layer.bias

    
    def forward(self, x):
        return self.bias + torch.mm(x, self.weights)

In [109]:
sl.bias  + (input_sparse @ sl.weights)

tensor([[-0.0738, -0.0234,  0.0346,  ..., -0.0094,  0.0154, -0.0011],
        [-0.1085,  0.0009,  0.0709,  ..., -0.0308,  0.0147,  0.0152],
        [-0.0445,  0.0006,  0.0638,  ..., -0.0376,  0.0010, -0.0057],
        ...,
        [-0.1035,  0.0213,  0.0500,  ...,  0.0013,  0.0042,  0.0554],
        [-0.0812,  0.0079,  0.0342,  ..., -0.0241, -0.0153,  0.0018],
        [-0.0975,  0.0019,  0.0885,  ..., -0.0015, -0.0043,  0.0365]],
       grad_fn=<AddBackward0>)

In [110]:
input = torch.rand(size=(64, 1024))
input_sparse = input.to_sparse_coo()
with torch.no_grad():
    sl = SparseLinear(sparse_linear, bs=64)
    start = datetime.now()
    for _ in range(100):
        out = sl(input_sparse)
    end = datetime.now() - start
    print(end)
out.shape
print(type(out))

0:00:00.544879
<class 'torch.Tensor'>


In [20]:
with torch.no_grad():
    input.to_sparse_coo() @ sparse_linear.weight.T.to_sparse_coo()

In [10]:
w = torch.tensor(
        [
            [
                1,2,3
            ],
            [
                4,5,6,
            ],
        ]
    )
w.shape
x = torch.tensor([[1,2],[3,4],[5,6],])
x.shape

torch.Size([3, 2])